# L5.2 Dashboards
Angel Johnattan Gil herrera
21051432

## Tema
Información crediticia de prestatarios, con el objetivo de predecir si una persona tendrá una morosidad grave (90+ días de atraso) en los próximos dos años. El análisis permite detectar factores de riesgo para decisiones financieras responsables."

In [1]:
# Importamos librerias
## librerias basicas
import pandas as pd
import numpy as np

## Modulos para generar el panel
import panel as pn
pn.extension('tabulator')
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

In [2]:
## Lectura del dataset y comprobacion de funcionamiento de lectura
df = pd.read_csv('Credit Risk Benchmark Dataset.csv')
df.head(20)

,rev_util,age,late_30_59,debt_ratio,monthly_inc,open_credit,late_90,real_estate,late_60_89,dependents,dlq_2yrs
0,0.006999,38.0,0.0,0.302150,5440.0,4.0,0.0,1.0,0.0,3.0,0
1,0.704592,63.0,0.0,0.471441,8000.0,9.0,0.0,1.0,0.0,0.0,0
2,0.063113,57.0,0.0,0.068586,5000.0,17.0,0.0,0.0,0.0,0.0,0
3,0.368397,68.0,0.0,0.296273,6250.0,16.0,0.0,2.0,0.0,0.0,0
4,1.000000,34.0,1.0,0.000000,3500.0,0.0,0.0,0.0,0.0,1.0,0
5,0.051799,63.0,0.0,0.770687,4700.0,16.0,0.0,1.0,0.0,1.0,0
6,0.648733,40.0,1.0,0.402060,6600.0,10.0,0.0,2.0,0.0,3.0,0
7,0.767270,33.0,0.0,0.338390,5800.0,10.0,0.0,0.0,0.0,3.0,0
8,0.190111,27.0,0.0,0.164605,3960.0,6.0,0.0,0.0,0.0,1.0,0
9,0.139989,65.0,1.0,0.258397,6400.0,6.0,0.0,1.0,0.0,0.0,0


## Informacion de columnas
Este dataset contiene lo siguiente
1) rev_util: Es cuanto credito disponible esta usando una persona en sus tarjetas. Alto valor = posible uso excesivo del crédito.
2) Age: Edad util para las correlaciones con estabilidad financiera
3) late_30_59, late_60_89, late_90
- Número de veces que la persona se ha retrasado en pagos.
- Son indicadores de morosidad pasada:
    * 30-59 días: leve
    * 60-89 días: moderada
    * 90+ días: grave
4) debt_ratio:
- Qué tan endeudada está una persona comparado con su ingreso.
- Alto valor puede ser una señal de riesgo
5) monthly_inc (Ingreso mensual): Mayor ingreso puede asociarse a menor probabilidad de morosidad (aunque no siempre)
6) open_credit:
- Cuántas cuentas de crédito/loans tiene abiertas
- Puede mostrar actividad crediticia o sobreexposición
7) real_estate:
- Número de hipotecas u otros préstamos de bienes raíces
- Relevante para evaluar patrimonio o cargas crediticias
8) dependents:
- Número de personas que dependen del prestatario.
- Más dependientes puede significar más presión financiera

In [3]:
df = df.rename(columns={
    'rev_util':'uso_de_credito',
    'age':'edad',
    'debt_ratio':'radio_deuda',
    'real_estate':'num_otros_prestamos',
    'dependents':'num_dependientes',
    'late_30_59':'atraso_30_59',
    'late_60_89':'atraso_60_89',
    'late_90':'atraso_90',
    'monthly_inc':'ingreso_mensual',
    'open_credit':'cuentas_abiertas'
})
dff = df.copy()
df.head()

,uso_de_credito,edad,atraso_30_59,radio_deuda,ingreso_mensual,cuentas_abiertas,atraso_90,num_otros_prestamos,atraso_60_89,num_dependientes,dlq_2yrs
0,0.006999,38.0,0.0,0.302150,5440.0,4.0,0.0,1.0,0.0,3.0,0
1,0.704592,63.0,0.0,0.471441,8000.0,9.0,0.0,1.0,0.0,0.0,0
2,0.063113,57.0,0.0,0.068586,5000.0,17.0,0.0,0.0,0.0,0.0,0
3,0.368397,68.0,0.0,0.296273,6250.0,16.0,0.0,2.0,0.0,0.0,0
4,1.000000,34.0,1.0,0.000000,3500.0,0.0,0.0,0.0,0.0,1.0,0


In [4]:
# Crear columna 'nivel_riesgo' según el atraso_90
def clasificar_riesgo(row):
    if row['atraso_90'] > 0:
        return 'Alto'
    elif row['atraso_60_89'] > 0:
        return 'Medio'
    elif row['atraso_30_59'] > 0:
        return 'Bajo'
    else:
        return 'Sin riesgo'

df['nivel_riesgo'] = df.apply(clasificar_riesgo, axis=1)
df.head()

,uso_de_credito,edad,atraso_30_59,radio_deuda,ingreso_mensual,cuentas_abiertas,atraso_90,num_otros_prestamos,atraso_60_89,num_dependientes,dlq_2yrs,nivel_riesgo
0,0.006999,38.0,0.0,0.302150,5440.0,4.0,0.0,1.0,0.0,3.0,0,Sin riesgo
1,0.704592,63.0,0.0,0.471441,8000.0,9.0,0.0,1.0,0.0,0.0,0,Sin riesgo
2,0.063113,57.0,0.0,0.068586,5000.0,17.0,0.0,0.0,0.0,0.0,0,Sin riesgo
3,0.368397,68.0,0.0,0.296273,6250.0,16.0,0.0,2.0,0.0,0.0,0,Sin riesgo
4,1.000000,34.0,1.0,0.000000,3500.0,0.0,0.0,0.0,0.0,1.0,0,Bajo


In [5]:
# Definir grupos de edad de 10 en 10
grupos_edad = []
for inicio in range(20, int(df['edad'].max())+1, 10):
    fin = inicio + 9
    grupos_edad.append(f"{inicio}-{fin}")

selector_grupos_edad = pn.widgets.MultiChoice(
    name='Grupos de edad',
    options=grupos_edad,
    value=[grupos_edad[0]]  # Solo el primer grupo seleccionado por default
)

In [6]:
# Widgets de filtro
rango_deuda = pn.widgets.RangeSlider(
    name='Rango radio_deuda', 
    start=float(df['radio_deuda'].min()), 
    end=float(df['radio_deuda'].max()), 
    value=(float(df['radio_deuda'].min()), float(df['radio_deuda'].max()))
)
selector_riesgo = pn.widgets.MultiChoice(
    name='Nivel de riesgo', 
    options=list(df['nivel_riesgo'].unique()), 
    value=list(df['nivel_riesgo'].unique())
)
max_dependientes = int(df['num_dependientes'].max())
selector_dependientes = pn.widgets.IntSlider(
    name='Número de dependientes',
    start=0,
    end=max_dependientes,
    value=0
)
edad_min = int(df['edad'].min())
edad_max = int(df['edad'].max())
rango_edad = pn.widgets.RangeSlider(
    name='Rango de edad',
    start=edad_min,
    end=edad_max,
    value=(edad_min, edad_max),
    step=1
)

In [7]:
# Función de filtrado
def filtrar_df(rango, riesgos, num_dep, rango_edad_val):
    if num_dep < 0 or num_dep > max_dependientes:
        return None
    return df[
        (df['radio_deuda'] >= rango[0]) & 
        (df['radio_deuda'] <= rango[1]) & 
        (df['nivel_riesgo'].isin(riesgos)) &
        (df['num_dependientes'] == num_dep) &
        (df['edad'] >= rango_edad_val[0]) &
        (df['edad'] <= rango_edad_val[1])
    ]


In [8]:
# Indicadores principales
@pn.depends(rango_deuda, selector_riesgo, selector_dependientes, rango_edad)
def indicadores(rango, riesgos, num_dep, rango_edad_val):
    dff = filtrar_df(rango, riesgos, num_dep, rango_edad_val)
    if dff is None or dff.empty:
        return pn.pane.Markdown("**No aplica**")
    total = len(dff)
    ingreso_promedio = dff['ingreso_mensual'].mean()
    return pn.Column(
        pn.pane.Markdown(f"### Total de registros\n# {total}"),
        pn.pane.Markdown(f"### Ingreso mensual promedio\n# {ingreso_promedio:,.2f}")
    )

In [9]:
# Gráfica: Relación edad-ingreso normalizada por nivel de riesgo
@pn.depends(rango_deuda, selector_riesgo, selector_dependientes, selector_grupos_edad)
def grafica_barras_edad_ingreso(rango, riesgos, num_dep, grupos_sel):
    bins = [int(g.split('-')[0]) for g in grupos_edad] + [int(grupos_edad[-1].split('-')[1])+1]
    labels = grupos_edad
    dff = filtrar_df(rango, riesgos, num_dep, (bins[0], bins[-1]-1))
    if dff is None or dff.empty or not grupos_sel:
        return pn.pane.Markdown("**No aplica**")
    dff = dff.copy()
    dff['grupo_edad'] = pd.cut(dff['edad'], bins=bins, labels=labels, right=True, include_lowest=True)
    dff = dff[dff['grupo_edad'].isin(grupos_sel)]
    # Ordenar el DataFrame por el orden de grupos_sel
    dff['grupo_edad'] = pd.Categorical(dff['grupo_edad'], categories=grupos_sel, ordered=True)
    tabla = dff.groupby(['grupo_edad', 'nivel_riesgo'], observed=False)['ingreso_mensual'].mean().reset_index()
    tabla = tabla.sort_values('grupo_edad')
    return tabla.hvplot.bar(
        x='grupo_edad',
        y='ingreso_mensual',
        by='nivel_riesgo',
        stacked=False,
        xlabel='Grupo de edad',
        ylabel='Ingreso mensual promedio',
        title='Ingreso mensual promedio por grupo de edad y nivel de riesgo',
        height=350,
        width=600,
        legend='top_right'
    )

@pn.depends(rango_deuda)
def grafica_tendencia_endeudamiento(rango):
    dff = df[
        (df['radio_deuda'] >= rango[0]) & 
        (df['radio_deuda'] <= rango[1])
    ].copy()
    if dff.empty:
        return pn.pane.Markdown("**No aplica**")
    # Calcular promedio de radio de deuda por número de dependientes
    tabla = dff.groupby('num_dependientes', observed=False)['radio_deuda'].mean().reset_index()
    # Normalizar a escala 1-10
    tabla['indice_endeudamiento'] = 1 + 9 * (tabla['radio_deuda'] - tabla['radio_deuda'].min()) / (tabla['radio_deuda'].max() - tabla['radio_deuda'].min())
    tabla['indice_endeudamiento'] = tabla['indice_endeudamiento'].round(1)
    return tabla.hvplot.bar(
        x='num_dependientes',
        y='indice_endeudamiento',
        color=hv.Cycle(['#800080']),  # Morado
        xlabel='Número de dependientes',
        ylabel='Índice de endeudamiento (1-10)',
        title='Tendencia de endeudamiento según número de dependientes',
        ylim=(1, 10),
        height=350,
        width=600,
        legend=None
    )

@pn.depends(rango_deuda)
def grafica_ingreso_vs_riesgo(rango):
    dff = df[
        (df['radio_deuda'] >= rango[0]) & 
        (df['radio_deuda'] <= rango[1])
    ].copy()
    if dff.empty:
        return pn.pane.Markdown("**No aplica**")
    tabla = dff.groupby('nivel_riesgo', observed=False)['ingreso_mensual'].mean().reset_index()
    tabla = tabla.sort_values('nivel_riesgo')
    return tabla.hvplot.bar(
        x='nivel_riesgo',
        y='ingreso_mensual',
        color='#800080',  # Morado
        xlabel='Nivel de riesgo',
        ylabel='Ingreso mensual promedio',
        title='Ingreso mensual promedio por nivel de riesgo',
        height=350,
        width=400,
        legend=None
    )

@pn.depends(rango_deuda)
def grafica_credito_vs_riesgo(rango):
    dff = df[
        (df['radio_deuda'] >= rango[0]) & 
        (df['radio_deuda'] <= rango[1])
    ].copy()
    if dff.empty:
        return pn.pane.Markdown("**No aplica**")
    tabla = dff.groupby('nivel_riesgo', observed=False)['uso_de_credito'].mean().reset_index()
    tabla = tabla.sort_values('nivel_riesgo')
    return tabla.hvplot.bar(
        x='nivel_riesgo',
        y='uso_de_credito',
        color='#800080',  # Morado
        xlabel='Nivel de riesgo',
        ylabel='Uso de crédito promedio',
        title='Uso de crédito promedio por nivel de riesgo',
        height=350,
        width=400,
        legend=None
    )

In [10]:
# Tabla interactiva
@pn.depends(rango_deuda, selector_riesgo, selector_dependientes, rango_edad)
def tabla(rango, riesgos, num_dep, rango_edad_val):
    dff = filtrar_df(rango, riesgos, num_dep, rango_edad_val)
    if dff is None or dff.empty:
        return pn.pane.Markdown("**No aplica**")
    return dff.head(20).pipe(pn.widgets.Tabulator, pagination='remote', page_size=10)

# Explicación del dashboard
explicacion = pn.pane.Markdown("""
## Dashboard de Riesgo Crediticio

Este panel permite analizar la información crediticia de los prestatarios:
- **Filtros:** Selecciona el rango de radio de deuda, el nivel de riesgo y el número de dependientes.
- **Nivel de riesgo:** Se calcula según los atrasos en pagos (mayor atraso, mayor riesgo).
- **Gráficas:** Observa la relación entre edad e ingreso (normalizados) por nivel de riesgo y el promedio de deuda según dependientes.
- **Tabla:** Consulta los detalles de los registros filtrados.
""")


In [13]:
# Diccionario de gráficas disponibles
graficas = {
    "Edad vs Ingreso (por riesgo)": grafica_barras_edad_ingreso,
    "Tendencia de endeudamiento": grafica_tendencia_endeudamiento,
    "Ingreso vs Riesgo": grafica_ingreso_vs_riesgo,
    "Uso de crédito vs Riesgo": grafica_credito_vs_riesgo
}

# Crear menú de pestañas para las gráficas
menu_graficas = pn.Tabs(
    *[(nombre, grafica) for nombre, grafica in graficas.items()],
    dynamic=True
)

# Nuevo layout del dashboard con menú de gráficas
dashboard = pn.Column(
    explicacion,
    pn.Row(
        pn.Column(
            pn.pane.Markdown("### Filtros"),
            #rango_deuda,
            selector_riesgo,
            selector_dependientes,
            selector_grupos_edad,
            indicadores
        ),
        pn.Column(
            menu_graficas  # Menú de gráficas con pestañas
        )
    ),
    pn.Row(
        pn.Column(
            pn.pane.Markdown("### Tabla de datos agregados"),
            tabla
        )
    )
)

In [14]:
dashboard.servable()

Column
    [0] Markdown(str)
    [1] Row
        [0] Column
            [0] Markdown(str)
            [1] MultiChoice(name='Nivel de riesgo', options=['Sin riesgo', ...], value=['Sin riesgo', ...])
            [2] IntSlider(end=8, name='Número de dependientes')
            [3] MultiChoice(name='Grupos de edad', options=['20-29', '30-39', ...], value=['20-29'])
            [4] ParamFunction(function, _pane=Column, defer_load=False)
        [1] Column
            [0] Tabs(dynamic=True)
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False, name='Edad vs Ingreso (...)
                [1] ParamFunction(function, _pane=HoloViews, defer_load=False, name='Tendencia de e...)
                [2] ParamFunction(function, _pane=HoloViews, defer_load=False, name='Ingreso vs Riesgo')
                [3] ParamFunction(function, _pane=HoloViews, defer_load=False, name='Uso de crédito v...)
    [2] Row
        [0] Column
            [0] Markdown(str)
            [1] ParamFunction(function, _pane=Tabulator, defer_load=False)